In [1]:
%run __init__.py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from itertools import islice
from itertools import combinations
from aper_phot import *
from cr2others import *

In [ ]:
def generate_src_reg(src_reg_name, 
                     r_low, r_up, r_step, 
                     d_low, d_up, d_step, 
                     dire_1, dire_2):
    # generate radiu list (array)
    radiu_list = np.linespace(r_low, r_up, r_step).tolist()
    # generate centers
    # dire_1 = [448, 1210]; dire_2 = [990, 1810]
    dire_dist = get_dist(dire_1, dire_2)
    dire_i = abs(dire_1[1]-dire_2[1])/dire_dist
    dire_j = abs(dire_1[0]-dire_2[0])/dire_dist
    dist_list = np.linespace(d_low, d_up, d_step).tolist()
    center_nucleus = [1001, 1001]
    center_i = center_nucleus[1] + dist_list*dire_i
    center_j = center_nucleus[0] + dist_list*dire_j
    center_list = list(zip(center_j, center_i))
    # write into a file
    src_reg_path = get_path('../docs/'+src_reg_name)
    f = open(src_reg_path, 'w')
    f.write('# Generated regions for photometry of the source'+'\n')
    f.write('global color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1'+'\n')
    f.write('physical'+'\n')
    for r in radiu_list:
        for c in center_list:
            f.write('circle(' + str(c[0]) + ',' + str(c[1]) + ',' + str(r)+') # color=black width=1' + '\n')
    f.close()

In [ ]:
def src_snr(img_name, reg_name, bg_method, bg_center, bg_r):
    # bkg -> bg_bri, bg_bri_err
    bg_bri, bg_bri_err = reg2bg_bri(img_name, method, bg_center, bg_r)
    # load regions
    center_i_list, center_j_list, radiu_list = load_reg_list(reg_name)
    # photometry -> snr
    dist_list = []
    count_list = []
    pixel_list = []
    cr_list = []
    snr_list = []
    exposure = load_header(img_name)['EXPTIME']
    reg_num = len(radiu_list)
    for n in range(reg_num):
        center = [center_i_list[n], center_j_list[n]]
        count, pixel = circle_ct(img_name, center, radiu_list[n])
        src_count_err = np.sqrt(count)
        cr, cr_err, snr = aper_phot(count,
                                    pixel,
                                    bg_bri, exposure,
                                    src_count_err,
                                    bg_bri_err)
        dist_list.append(get_dist(center, (1001, 1001)))
        count_list.append(count)
        pixel_list.append(pixel)
        cr_list.append(cr)
        snr_list.append(snr)
    # plot snr-size, snr-dist
    fig = plt.figure()
    ax1 = fig.add_subplot(121)
    ax1.plot(radiu_list, snr_list, fmt='r')
    ax2 = fig.add_subplot(122)
    plt.plot(dist_list, snr_list, fmt='b')
    plt.show()
    # plot 2D, ref: stack overflow: 48708862, 39727040

In [ ]:
def bg_snr(method, img_name, reg_name, src_center, src_r):
    # source photometry
    exposure = load_header(img_name)['EXPTIME']
    src_count, src_pixel = circle_ct(img_name, src_center, src_r)
    src_count_err = np.sqrt(src_count)
    # load regions
    center_i_list, center_j_list, radiu_list = load_reg_list(reg_name)
    center_list = list(zip(center_j_list, center_i_list))
    # photometry -> snr
    snr_list = []
    cr_list = []
    for n in range(len(radiu_list)):
        bg_bri, bg_bri_err = reg2bg_bri(img_name, method, center_list[n], r_list[n])
        cr, cr_err, snr = aper_phot(src_count,
                                    src_pixel,
                                    bg_bri, exposure, 
                                    src_count_err,
                                    bg_bri_err)
        snr_list.append(snr)
        cr_list.append(snr)
    # plot snr-size
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(radiu_list, snr_list)
    plt.show()
    # plot bubble map: 14827650

In [ ]:
def bg_multi_circle_snr(img_name, reg_name, src_center, src_r, n):
    # source photometry
    exposure = load_header(img_name)['EXPTIME']
    src_count, src_pixel = circle_ct(img_name, src_center, src_r)
    src_count_err = np.sqrt(src_count)
    # load regions to lists of x, y and r
    center_i_list, center_j_list, radiu_list = load_reg_list(reg_name)
    radiu_set = set(radiu_list)
    # convert 3 lists to dataframe
    bg_reg = {'x': center_j_list,
              'y': center_i_list,
              'r': radiu_list}
    bg_reg = pd.DataFrame(bg_regions)
    # loop for every r and n locations, note: check Exception
    cr_list = []
    snr_list = []
    size_list = []
    distribution_list = []
    distance_list = []
    for r in radiu_set:
        center_df = bg_reg(bg_reg['r']==r)
        if len(center_df) < n:
            raise Exception('number of bg regions is less than n for radiu-'+str(r))
        center_x = list(center_df['x'])
        center_y = list(center_df['y'])
        center_for_r = list(zip(center_x, center_y))
        center_group = combinations(center_for_r, n)
        # get the distribution value and distances to the source
        for center_list in center_group:
            r_list = [r for k in range(n)]
            # photometry
            count_list, pixel_list = multi_circle_ct(img_name, center_list, r_list)
            bg_bri, bg_bri_err = reg2bg_bri(img_name, 'multi', center_list, r_list, n)
            cr, cr_err, snr = aper_phot(src_count, src_pixel, bg_bri, exposure, src_count_err, bg_bri_err)
            # output - cr, snr, size
            cr_list.append(cr)
            snr_list.append(snr)
            size_list.append(sum(pixel_list))
            # output - distribution
            distribution = 0
            for line in combination(center_list, 2):
                distribution += get_dist(line[0], line[1])
            distribution = distribition/len(combination(center_list, 2))
            distribution_list.append(distribution)
            # output - mean distance
            dist = 0
            for center in center_list:
                dist += get_dist(center, src_center)
            dist = dist/len(center_list)
            distance_list.append(dist)
    # plot
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(size_list, snr_list)
    plt.show()
    # photometry -> snr